In [1]:
# authorize access to the drive
# This allows me to access files (Data) in my google drive
from google.colab import drive
drive.mount("/content/drive")


Mounted at /content/drive


In [2]:
# This downloads Hugging Face Transformers, tools we need to fine-tune GPT-2
# need an older version of transformers
!pip3 install transformers==3.3.1
#!pip3 install git+https://github.com/huggingface/transformers@3.3.1
#!pip3 install git+https://github.com/huggingface/transformers

     |████████████████████████████████| 1.1MB 7.7MB/s 
     |████████████████████████████████| 1.2MB 27.6MB/s 
     |████████████████████████████████| 890kB 48.2MB/s 
     |████████████████████████████████| 3.0MB 51.3MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893261 sha256=9dff9a09f9b494bff4cd2125a794ff464ea126eb58fd03d9a7f8cf4fa9cc1d44
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [3]:
# Allows me to go to the correct directory in my google drie
%cd drive/My\ Drive/School\ Work/Fourth\ Year/Capstone\ Everything

/content/drive/My Drive/School Work/Fourth Year/Capstone Everything


In [4]:
# the fine-tuning code by Hugging Face that allows us to fine-tune
# this file is already archieved after the most recent hugging face modelling se we need to replace the url
# !wget https://raw.githubusercontent.com/huggingface/transformers/master/examples/language-modeling/run_language_modeling.py
!wget https://raw.githubusercontent.com/huggingface/transformers/master/examples/contrib/legacy/run_language_modeling.py

--2021-02-11 07:22:50--  https://raw.githubusercontent.com/huggingface/transformers/master/examples/contrib/legacy/run_language_modeling.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 404 Not Found
2021-02-11 07:22:50 ERROR 404: Not Found.



In [5]:
% ls

'Brian'\''s Peer Feedback Template (10.28.21).gdoc'
'Capstone Coversheet.gdoc'
'Capstone Proposal (Junior Year).gdoc'
 class/
'Dev Journal.gdoc'
 Experiments/
 Models/
 new_run_language_modeling.py
 None/
'Paper Drafts'/
'Punch Card.gsheet'
 run_language_modeling.py
 run_language_modeling.py.1
 runs/


In [6]:
# get dataset
datapath = "Models/Data"
train_path = datapath + "/abstract_train.txt"
valid_path = datapath + "/abstract_valid.txt"
output = "./output"

In [ ]:
# %%bash 
# export DIRECTORY=Models/Data/Description_Data
# ls $DIRECTORY

In [7]:
# Main fine-tuning code using run_language_modeling.py
%%bash
export TRAIN_FILE=Models/Data/abstract_train.txt
export TEST_FILE=Models/Data/abstract_valid.txt
export MODEL_NAME=gpt2
export OUTPUT_DIR=Models/Experiments/Abstracts/20epochs

python run_language_modeling.py \
    --output_dir=$OUTPUT_DIR \
    --model_type=gpt2 \
    --model_name_or_path=$MODEL_NAME \
    --do_train \
    --train_data_file=$TRAIN_FILE \
    --do_eval \
    --eval_data_file=$TEST_FILE \
    --per_gpu_train_batch_size=1 \
    --save_steps=-1 \
    --num_train_epochs=20

{'loss': 1.60418896484375, 'learning_rate': 4.89451476793249e-05, 'epoch': 0.4219409282700422, 'total_flos': 382279090176000, 'step': 500}
{'loss': 0.957919921875, 'learning_rate': 4.789029535864979e-05, 'epoch': 0.8438818565400844, 'total_flos': 764558180352000, 'step': 1000}
{'loss': 0.56267724609375, 'learning_rate': 4.683544303797468e-05, 'epoch': 1.2658227848101267, 'total_flos': 1146837270528000, 'step': 1500}
{'loss': 0.356543212890625, 'learning_rate': 4.5780590717299585e-05, 'epoch': 1.6877637130801688, 'total_flos': 1529116360704000, 'step': 2000}
{'loss': 0.2426953125, 'learning_rate': 4.4725738396624474e-05, 'epoch': 2.109704641350211, 'total_flos': 1911395450880000, 'step': 2500}
{'loss': 0.1794169921875, 'learning_rate': 4.367088607594937e-05, 'epoch': 2.5316455696202533, 'total_flos': 2293674541056000, 'step': 3000}
{'loss': 0.150966064453125, 'learning_rate': 4.2616033755274265e-05, 'epoch': 2.9535864978902953, 'total_flos': 2675953631232000, 'step': 3500}
{'loss': 0.12

2021-02-11 07:23:05.734190: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
/usr/local/lib/python3.6/dist-packages/transformers/training_args.py:332: FutureWarning: The `evaluate_during_training` argument is deprecated in favor of `evaluation_strategy` (which has more options)
  FutureWarning,
02/11/2021 07:23:08 - WARNING - __main__ -   Process rank: -1, device: cuda:0, n_gpu: 1, distributed training: False, 16-bits training: False
02/11/2021 07:23:08 - INFO - __main__ -   Training/evaluation parameters TrainingArguments(output_dir='Models/Experiments/Abstracts/20epochs', overwrite_output_dir=False, do_train=True, do_eval=True, do_predict=False, evaluate_during_training=False, evaluation_strategy=<EvaluationStrategy.NO: 'no'>, prediction_loss_only=False, per_device_train_batch_size=8, per_device_eval_batch_size=8, per_gpu_train_batch_size=1, per_gpu_eval_batch_size=None, gradient_accumulation_steps=1, learning_rate=

In [ ]:
# this allow us to generate sample patent claims 

from transformers import GPT2Tokenizer, GPT2LMHeadModel
import torch
import numpy as np

OUTPUT_DIR = "./Models/Experiments/Abstracts/20epochs"
device = 'cpu'
if torch.cuda.is_available():
    device = 'cuda'
# we use the GPT-2 model to load our fine-tuned model and get the tokenization
tokenizer = GPT2Tokenizer.from_pretrained(OUTPUT_DIR)
model = GPT2LMHeadModel.from_pretrained(OUTPUT_DIR)
model = model.to(device)
                                        
def generate(input_str, length=250, n=5):
  '''
  This is the main generation code using our model
  '''
  cur_ids = torch.tensor(tokenizer.encode(input_str)).unsqueeze(0).long().to(device)
  model.eval()
  with torch.no_grad():
    for i in range(length):
      outputs = model(cur_ids[:, -1024:], labels=cur_ids[:, -1024:])
      loss, logits = outputs[:2]
      softmax_logits = torch.softmax(logits[0,-1], dim=0)
      next_token_id = choose_from_top(softmax_logits.to('cpu').numpy(), n=n)
      cur_ids = torch.cat([cur_ids, torch.ones((1,1)).long().to(device) * next_token_id], dim=1)
    output_list = list(cur_ids.squeeze().to('cpu').numpy())
    output_text = tokenizer.decode(output_list)
    return output_text

def choose_from_top(probs, n=5):
    '''
    This chooses the top n choices and probabilistically select one choice
    '''
    ind = np.argpartition(probs, -n)[-n:]
    top_prob = probs[ind]
    top_prob = top_prob / np.sum(top_prob) # Normalize
    choice = np.random.choice(n, 1, p = top_prob)
    token_id = ind[choice][0]
    return int(token_id)
# print the generated text
generated_text = generate("<BOS>", n=5)
print(generated_text)

<BOS> Provided are a compound represented by Formula 1, and an organic electric element comprising a first electrode, a second electrode, and an organic material layer      between the first electrode and the second electrode, wherein the organic material      layer comprises the compound represented by Formula 1, and the driving voltage of an organic      electronic device can be lowered, and the luminous efficiency, color purity and life time      of an organic electronic device can be improved by comprising the compound      represented by Formula 1 in the organic material layer. <EOS>
<BOS> A semiconductor device includes a first semiconductor die stacked over a      second semiconductor die on a substrate and including an active region      and a peripheral region. A first region is separated from the first region by at      least a first opening, a second region is separated from the first      region by at least a second opening, and a through region is      disposed between the

In [ ]:
def trim_string(row):

    row = row.split(maxsplit=first_n_words)
    row = ' '.join(row[:first_n_words])

    return row

def trim_abstract(row): 
  # just needs to take out the <EOS> <BOS> tags
  row = row.replace('<EOS>', '')
  row = row.replace('<BOS>', '')
  return row

In [ ]:
first_n_words = 250
trimmed_text = trim_string(trim_abstract(generated_text))
trimmed_text

'Provided are a compound represented by Formula 1, and an organic electric element comprising a first electrode, a second electrode, and an organic material layer between the first electrode and the second electrode, wherein the organic material layer comprises the compound represented by Formula 1, and the driving voltage of an organic electronic device can be lowered, and the luminous efficiency, color purity and life time of an organic electronic device can be improved by comprising the compound represented by Formula 1 in the organic material layer. A semiconductor device includes a first semiconductor die stacked over a second semiconductor die on a substrate and including an active region and a peripheral region. A first region is separated from the first region by at least a first opening, a second region is separated from the first region by at least a second opening, and a through region is disposed between the first opening and the second opening. The first and second semicon